In [ ]:
"""
Hello y'all see this?
"""
from google.colab import drive
import random
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Connect Colab to Google Drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Unzip training data, run once
#!unzip drive/MyDrive/data/train.zip -d drive/MyDrive/data/

In [ ]:
# Unzip validation data, run once
#!unzip drive/MyDrive/data/validation.zip -d drive/MyDrive/data/

In [7]:
# This code chunk puts all image paths and their labels in a dataframe
train_paths = []
train_labels = []
validation_paths = []
validation_labels = []
troot = 'drive/MyDrive/data/train-cropped'
vroot = 'drive/MyDrive/data/validation-cropped'

for f in os.listdir(troot):
  tpath = os.path.join(troot, f)
  tlabel = f[-10:-8]
  train_paths.append(tpath)
  train_labels.append(tlabel)

for f in os.listdir(vroot):
  vpath = os.path.join(vroot, f)
  vlabel = f[-10:-8]
  validation_paths.append(vpath)
  validation_labels.append(vlabel)

train_images = pd.DataFrame({'path': train_paths,
                             'label': train_labels})
validation_images = pd.DataFrame({'path': validation_paths,
                                  'label': validation_labels})

In [8]:
def get_subset(n, data, seed=123):
  random.seed(seed)
  sub = random.sample(range(data.shape[0] + 1), n)
  return data.iloc[sub,:]

train_subset = get_subset(1000, train_images)
validation_subset = get_subset(1000, validation_images)

In [15]:
imageWidth, imageHeight = 128, 128
channel = 1
imageSize = (imageWidth, imageHeight)

In [21]:
train_generator = ImageDataGenerator(
    rescale=1./255
).flow_from_dataframe(
    dataframe = train_subset,
    directory = None,         # `train_subset` contains absolute paths
    x_col='path',
    y_col='label',
    class_mode='categorical',  # target is 2-D array of one-hot encoded labels
    target_size=imageSize,
    color_mode="grayscale",
    shuffle=False
    )

validation_generator = ImageDataGenerator(
    rescale=1./255
).flow_from_dataframe(
    dataframe = validation_subset,
    directory = None,
    x_col='path',
    class_mode= None,
    target_size=imageSize,
    color_mode="grayscale",
    shuffle=False
    )

Found 1000 validated image filenames belonging to 5 classes.
Found 1000 validated image filenames.


In [24]:
nCategories = 5
model = Sequential()
## First convolutional layer with 32 filters (kernels)
model.add(Conv2D(filters = 32,
                 kernel_size=2,
                 activation='relu',
                 input_shape=(imageWidth, imageHeight, channel)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))
## 2nd convolutional layer
model.add(Conv2D(64,
                 kernel_size = 2,
                 activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))
## 3rd convolutional layer
model.add(Conv2D(filters = 128,
                 kernel_size=2,
                 activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))
## Flatten the image into a string of pixels
model.add(Flatten())
## Use one final dense layer
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
## Output layer with 2 softmax nodes
model.add(Dense(nCategories, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, epochs=2)
history.history
#model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#
#history = model.fit(train_generator, epochs=1)
#train_phat = model.predict(train_generator)
#validation_phat = model.predict(validation_generator)
#label_map = {0:"DA", 1:"EN", 2:"RU", 3:"TH", 4:"ZN"}
#
#train_predictions = np.argmax(train_phat, axis=-1)
#validation_predictions = np.argmax(validation_phat, axis=-1)
#
#train_true_labels = train_subset['label'].replace(label_map)
#validation_true_lables = train_subset['label'].replace(label_map)
#
#tacc = np.sum([1 if train_predictions[i] == train_true_labels[i] else 0 for i in range(len(train_predictions))])
#tacc

Epoch 1/2
32/32 [==============================] - 199s 6s/step - loss: 1.1102 - accuracy: 0.7260
Epoch 2/2
32/32 [==============================] - 25s 763ms/step - loss: 0.4806 - accuracy: 0.8450


{'loss': [1.1102466583251953, 0.4805636703968048],
 'accuracy': [0.7260000109672546, 0.8450000286102295]}

In [26]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_19 (Conv2D)          (None, 127, 127, 32)      160       
                                                                 
 batch_normalization_24 (Ba  (None, 127, 127, 32)      128       
 tchNormalization)                                               
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 63, 63, 32)        0         
 ng2D)                                                           
                                                                 
 dropout_24 (Dropout)        (None, 63, 63, 32)        0         
                                                                 
 conv2d_20 (Conv2D)          (None, 62, 62, 64)        8256      
                                                                 
 batch_normalization_25 (Ba  (None, 62, 62, 64)       